## **PySpark SQL - Getting Started Notebook**
This notebook provides an example for installing Spark dependencies and a simple "getting started" syntax for PySpark SQL.

NOTE: This notebook is designed to be used with Google's Colab notebook and the Python 3 runtime.

## **Install Spark Dependencies**



In [ ]:
# Install Spark dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget --no-cookies --no-check-certificate https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar zxvf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark

In [ ]:
!ls -al | grep spark

## **Set env variables within Pyspark**

In [ ]:
# Set up required environment variables
import os
os.environ["JAVA_HOME"]  = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

## **Download Data**

In [ ]:
!wget https://raw.githubusercontent.com/zaratsian/iaa-2022/main/session_02/bikeshare_station_info.csv
!wget https://raw.githubusercontent.com/zaratsian/iaa-2022/main/session_02/bikeshare_trips.csv

## **Import Python and PySpark Libraries**

In [ ]:
import datetime
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import monotonically_increasing_id, col, expr, when, concat, lit, udf, split
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import GBTRegressor, LinearRegression, GeneralizedLinearRegression
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorIndexer, VectorAssembler, StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

## **Initialize Spark Session**

In [ ]:
spark = SparkSession.builder.appName("Bikesharing SparkSQL").master("local[*]").getOrCreate()

## **Read CSV into Spark**

In [ ]:
bikeshare_trips = spark.read.load('bikeshare_trips.csv', format="csv", header=True, inferSchema=True)

## **Display first few records**

In [ ]:
bikeshare_trips.show(5, truncate=False)

## **Execute Sample SparkSQL query**

In [ ]:
bikeshare_trips.createOrReplaceTempView("bikeshare_trips")
spark.sql("SELECT subscriber_type, count(*) as count FROM bikeshare_trips group by subscriber_type order by count desc").show(truncate=False)